In [15]:
# !pip install -r requirements.txt
# !python -m nltk.downloader stopwords
# !python -m nltk.downloader wordnet
# !python -m nltk.downloader omw-1.4

In [16]:
from nltk.corpus import stopwords
import snscrape.modules.telegram as telegram
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer
import pickle

In [17]:
scraper = telegram.TelegramChannelScraper('thebull_crypto')
telegram_df = pd.DataFrame()
for i,tel in enumerate(scraper.get_items()):
  if ( i < 100):
    df = pd.DataFrame([{'Date':tel.date,'Content':tel.content}])
    telegram_df = pd.concat([telegram_df,df],axis=0,ignore_index=True)
  else:
    break
telegram_df

,Date,Content
0,2022-07-20 10:52:35+00:00,$SKL already +20%
1,2022-07-20 07:38:22+00:00,#Bitcoin ✅
2,2022-07-20 07:34:41+00:00,Buying more $OGN here as well.
3,2022-07-20 07:30:37+00:00,$BAND still can buy!
4,2022-07-19 21:57:02+00:00,$BAND buying here!
...,...,...
95,2022-06-24 21:31:52+00:00,$LINA almost +20%
96,2022-06-24 21:12:34+00:00,$TRB seems ready to take off 🚀
97,2022-06-24 21:04:23+00:00,$SOL is moving nicely. Already +36% from entry!
98,2022-06-24 18:21:39+00:00,With only $30m mcap $TRB will go up +100% easily.


In [18]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
# current_date =date.today()
def text_cleaning(text):
    if(text!=None):
        stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        return normalized.split()

telegram_df["Cleaned Text"]= telegram_df["Content"].apply(text_cleaning)


In [19]:
def my_predictions(my_sentence, model):
    prediction = model.predict([my_sentence])
    return prediction

result=list()
model = pickle.load(open('./Telegram.pkl', 'rb'))
# telegram_df["Cleaned Text"].apply(lambda x: x.lower())
for i in telegram_df["Cleaned Text"]:
    result.append(my_predictions(str(i),model))
telegram_df["Result"]=result
telegram_df

,Date,Content,Cleaned Text,Result
0,2022-07-20 10:52:35+00:00,$SKL already +20%,"[skl, already, 20]",[2]
1,2022-07-20 07:38:22+00:00,#Bitcoin ✅,"[bitcoin, ✅]",[2]
2,2022-07-20 07:34:41+00:00,Buying more $OGN here as well.,"[buying, ogn, well]",[1]
3,2022-07-20 07:30:37+00:00,$BAND still can buy!,"[band, still, buy]",[1]
4,2022-07-19 21:57:02+00:00,$BAND buying here!,"[band, buying, here]",[1]
...,...,...,...,...
95,2022-06-24 21:31:52+00:00,$LINA almost +20%,"[lina, almost, 20]",[2]
96,2022-06-24 21:12:34+00:00,$TRB seems ready to take off 🚀,"[trb, seems, ready, take, 🚀]",[1]
97,2022-06-24 21:04:23+00:00,$SOL is moving nicely. Already +36% from entry!,"[sol, moving, nicely, already, 36, entry]",[2]
98,2022-06-24 18:21:39+00:00,With only $30m mcap $TRB will go up +100% easily.,"[30m, mcap, trb, go, 100, easily]",[1]


In [22]:
telegram_df.to_csv("Telegram_Predicted.csv")

In [21]:
# !pip freeze > requirements.txt